In [37]:
import requests
from hashlib import md5

class Chaojiying(object):
    def __init__(self, username, password, soft_id):
        self.username = username
        self.password = md5(password.encode('utf-8')).hexdigest()
        self.soft_id = soft_id
        self.base_params = {
            'user': self.username,
            'pass2': self.password,
            'softid': self.soft_id,
        }
        self.headers = {
            'Connection': 'Keep-Alive',
            'User-Agent': 'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0)',
        }

    def post_pic(self, im, codetype):
        """
        im: 图片字节
        codetype: 题目类型 参考 http://www.chaojiying.com/price.html
        """
        params = {
            'codetype': codetype,
        }
        params.update(self.base_params)
        files = {'userfile': ('ccc.jpg', im)}
        r = requests.post('http://upload.chaojiying.net/Upload/Processing.php', data=params, files=files,
                          headers=self.headers)
        return r.json()

    def report_error(self, im_id):
        """
        im_id:报错题目的图片ID
        """
        params = {
            'id': im_id,
        }
        params.update(self.base_params)
        r = requests.post('http://upload.chaojiying.net/Upload/ReportError.php', data=params, headers=self.headers)
        return r.json()

In [62]:
import random
import time
from io import BytesIO
import requests
from PIL import Image
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

USERNAME = '18562503270'
PASSWORD = 'wn2000123'

CHAOJIYING_USERNAME = 'hehaheha'
CHAOJIYING_PASSWORD = 'wn2000123'
CHAOJIYING_SOFT_ID = 915619
CHAOJIYING_KIND = 9004


class CrackTouClick():
    def __init__(self):
        self.url = 'https://login.zhipin.com/?ka=header-login'
        option = webdriver.ChromeOptions()
        option.add_experimental_option('excludeSwitches', ['enable-automation'])
        self.browser = webdriver.Chrome('chromedriver.exe',options=option)
        self.browser.maximize_window()
        self.wait = WebDriverWait(self.browser, 20)
        self.username = USERNAME
        self.password = PASSWORD


    def open(self):
        """
        打开网页输入用户名密码
        :return: None
        """
        self.browser.get(self.url)
        user = self.browser.find_element_by_xpath("//*[@id=\"wrap\"]/div[2]/div[1]/div[2]/div[1]/form/div[3]/span[2]/input")
        password = self.browser.find_element_by_xpath("//*[@id=\"wrap\"]/div[2]/div[1]/div[2]/div[1]/form/div[4]/span/input")
        user.send_keys(self.username)
        password.send_keys(self.password)
        login_btn = self.browser.find_element_by_xpath("//*[@id=\"pwdVerrifyCode\"]/div")
        # 随机暂停几秒
        time.sleep(random.random() * 3)
        # 点击登陆按钮
        login_btn.click()

    def pick_code(self):
        time.sleep(3)
        pick_img_label = self.browser.find_element_by_class_name('geetest_item_img')  # 获取点触图片标签
        src = pick_img_label.get_attribute('src')  # 获取点触图片链接
        img_content = requests.get(src).content  # 获取图片二进制内容
        f = BytesIO()
        f.write(img_content)
        img0 = Image.open(f)  # 将图片以文件的形式打开，主要是为了获取图片的大小
        scale = [pick_img_label.size['width'] / img0.size[0],
                 pick_img_label.size['height'] / img0.size[1]]  # 获取图片与浏览器该标签大小的比例
        cjy = Chaojiying(CHAOJIYING_USERNAME, CHAOJIYING_PASSWORD, CHAOJIYING_SOFT_ID)
        result = cjy.post_pic(img_content, '9005')  # 发送图片并获取结果
        if result['err_no'] == 0:  # 对结果进行分析
            position = result['pic_str'].split('|')  # position = ['110,234','145,247','25,185']
            position = [[int(j) for j in i.split(',')] for i in position]  # position = [[110,234],[145,247],[25,185]]
            for items in position:  # 模拟点击
                ActionChains(self.browser).move_to_element_with_offset(pick_img_label, items[0] * scale[0],
                                                                       items[1] * scale[1]).click().perform()
                time.sleep(1)
            time.sleep(2)
            # 点击登录
            certern_btn=self.browser.find_element_by_class_name("geetest_commit_tip")
            time.sleep(1)
            certern_btn.click()
        print(result)
        return cjy, result

    def crack(self):
        """
        破解入口
        :return: None
        """
        self.open()
        self.pick_code()
        btn=self.browser.find_element_by_xpath("//*[@id=\"wrap\"]/div[2]/div[1]/div[2]/div[1]/form/div[6]/button")
        btn.click()

In [4]:
from selenium import webdriver
import json
import time

class Cookies:
    def __init__(self, driver):
        self.driver = driver

    def save_cookies(self):
        cookies = self.driver.get_cookies()
        json_cookies = json.dumps(cookies)
        # 保存cookies，方便后续加载使用
        with open('cookies.json', 'w', encoding='utf-8') as f:
            f.write(json_cookies)

    def add_cookies(self):
        # 加载前先清除之前的cooikes，防止影响
        self.driver.delete_all_cookies()
        with open('cookies.json', 'r', encoding='utf-8') as f:
            list_cookies = json.loads(f.read())
        for i in list_cookies:
            self.driver.add_cookie(i)

In [5]:
import pymysql
from sqlalchemy import create_engine
'''
save_to_mysql方法将数据导入到mysql数据库里
'''
def save_to_mysql(data,table_name):
    engine = create_engine("mysql+pymysql://{}:{}@{}/{}?charset={}".format('root', 'welcome2019', '127.0.0.1:3306', 'WebDataOfTeachers','utf8mb4'))
    con = engine.connect()#创建连接
    data.to_sql(name=table_name, con=con, if_exists='append', index=False)#将dataframe导入到mysql数据库的表中。

In [6]:
'''
to_dataframe方法将数据转为dataframe，方便后续数据操作与整理
'''
import pandas
def to_dataframe(data):
    data=pandas.DataFrame(data)#将list类型的数据转为dataframe
    data=data[data['name']!=""]#去除name为空字符串的记录
    return data

In [7]:
def get_data(jobs):
    info=driver.find_elements_by_class_name("job-primary")
    for item in info:
        job={}
        
        job['name']=item.find_element_by_class_name("job-name").text
        job['address']=item.find_element_by_class_name("job-area-wrapper").text
        job['update']=item.find_element_by_class_name("job-pub-time").text
        job['wage']=item.find_element_by_class_name("red").text
        job['degree']=item.find_element_by_class_name("job-limit").find_element_by_tag_name("p").text
        job['company']=item.find_element_by_class_name("company-text").find_element_by_class_name("name").text
        job['company_info']=item.find_element_by_class_name("company-text").find_element_by_tag_name("p").text
        job['tag']=item.find_element_by_class_name("tags").text
        btn=item.find_element_by_class_name("job-title")
        btn.click()
        # 获取当前全部窗口句柄集合；循环句柄并判断是否与当前打开的页面一致，不一致就关闭并切换到一致的窗口句柄
        handles = driver.window_handles
        driver.switch_to.window(handles[1])
#         time.sleep(3)
        job['detail']=driver.find_element_by_class_name("job-sec").find_element_by_class_name("text").text
        driver.close()
        driver.switch_to.window(handles[0])
        time.sleep(1)
        jobs.append(job)

In [10]:
url = 'https://www.zhipin.com/job_detail/?ka=header-job'
option = webdriver.ChromeOptions()
# option.add_argument("headless")
option.add_experimental_option('excludeSwitches', ['enable-automation'])
driver = webdriver.Chrome('chromedriver.exe',options=option)
driver.maximize_window()

driver.implicitly_wait(30) # 隐性等待，最长等30秒
driver.get(url)
crack = CrackTouClick()
crack.crack()
cookies = Cookies(driver)
# 保存cookies
cookies.save_cookies()
# 加载cookies
cookies.add_cookies()
driver.refresh()
driver.get(url)


In [3]:
url=[
   
    "https://www.zhipin.com/c101110100-p101399/?ka=sel-city-101110100",
    "https://www.zhipin.com/c101190400-p101399/?ka=sel-city-101190400"
]

In [30]:
for i in url:
    driver.get(i)
    btn=driver.find_element_by_class_name("page").find_element_by_class_name("next")
    count=0
    while 1:
        jobs=[]
        get_data(jobs)
        data=to_dataframe(jobs)
        save_to_mysql(data,"jobs1")
        btn.click()
        time.sleep(3)
        btn=driver.find_element_by_class_name("page").find_element_by_class_name("next")
        count=count+1
        print(count)
        if count==10:
            break


1
2
3
4
5
1
2
3
4
5


{'err_no': 0, 'err_str': 'OK', 'pic_id': '1138622356109900008', 'pic_str': '283,212|230,62|54,239', 'md5': 'f4810dcd7f3610a6f4e667077aed1876'}
